# Some functions performing useful data preprocessing

In [2]:
import h5py
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import scipy
from scipy.spatial.distance import cdist

### Transform data from all cameras into a dataframe containing all features

In [6]:
hdf5data = h5py.File('../../Data/data_109.h5', 'r')

In [23]:
# np.asarray(hdf5data['Sequence'][str(0)]['Detections'])[0]

((-11.14840948, 15.36387945, -0.94192017), 1.17728508, 0.56196702, 1.33484852, 0.6319266)

In [30]:
# a = np.array([1,2,3,4,5])
# a[np.argmin(np.array([[1,2,3], [3,2, -1], [3,2,-1]]), axis=1)]

array([1, 3, 3])

In [3]:
def link_detections_tracks(detections, tracks):
    
    coord_detections = [np.array(vals[0].tolist()) for vals in detections]
    coord_tracks = [np.array(vals[1].tolist()) for vals in tracks]
    vel_tracks = [np.array(vals[2].tolist()) for vals in tracks]
    track_id = [vals[0].tolist() for vals in tracks]
    object_type = [vals[3] for vals in tracks]
    status_tracks = [vals[4] for vals in tracks]
    
    # for the time being, let us associate the closest bounding box, even if it is assigned more than once
    
    dists = cdist(coord_tracks, coord_detections, 'euclidean')
#     print(dists)
    associations = np.argmin(dists, axis=1)
    
    associations = associations.tolist()
    associated_detections = [detections[i] for i in associations]
    coord_detections = [coord_detections[i] for i in associations]
    length_box = [vals[1] for vals in associated_detections]
    width_box = [vals[2] for vals in associated_detections]
    height_box = [vals[3] for vals in associated_detections]
    angle_box = [vals[4] for vals in associated_detections]
    
    coord_detections = np.vstack(coord_detections)
    coord_tracks = np.vstack(coord_tracks)
    vel_tracks = np.vstack(vel_tracks)
    
    returndf = pd.DataFrame()
#     print(coord_tracks)
    returndf["X_track"] = coord_tracks[:,0]
    returndf["Y_track"] = coord_tracks[:,1]
    returndf["Z_track"] = coord_tracks[:,2]
#     print(returndf)
#     print(track_id)
    returndf["vx_track"] = vel_tracks[:,0]
    returndf["vy_track"] = vel_tracks[:,1]
    returndf["vz_track"] = vel_tracks[:,2]
    returndf["track_id"] = track_id
    returndf["obj_type"] = object_type
    returndf["track_status"] = status_tracks
    
    returndf["X_box"] = coord_detections[:,0]
    returndf["Y_box"] = coord_detections[:,1]
    returndf["Z_box"] = coord_detections[:,2]
    returndf["length_box"] = length_box
    returndf["width_box"] = width_box
    returndf["height_box"] = height_box
    returndf["angle_box"] = angle_box
    
    return returndf

In [10]:
def h5_to_df(h5data):
    
    timestamps = h5data['Timestamp']
    df = pd.DataFrame()
    
    for c, t in enumerate(timestamps):
        d = h5data['Sequence'][str(c)]
        track = np.asarray(d['Tracks'])
        detection = np.asarray(d['Detections'])
        row = link_detections_tracks(detection, track)
        append_df = link_detections_tracks(detection, track)
        append_df['timestamp'] = t
        append_df['frame'] = c
        df = df.append(append_df)
    return df

In [11]:
df = h5_to_df(hdf5data)

In [12]:
df.reset_index(inplace=True, drop=True)
df
df['camera'] = '109'

In [13]:
df

,X_track,Y_track,Z_track,vx_track,vy_track,vz_track,track_id,obj_type,track_status,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera
0,6.548051,26.367316,-0.847761,0.207261,0.359813,-0.045486,779,4,2,6.546980,26.314280,-0.862299,1.506848,0.592773,1.134711,0.424306,1.574333e+09,0,109
1,-11.134070,15.370831,-0.969107,0.393725,0.568159,0.052731,798,4,2,-11.148409,15.363879,-0.941920,1.177285,0.561967,1.334849,0.631927,1.574333e+09,0,109
2,-9.613478,18.832103,-1.370161,-0.963250,-1.160499,-0.050569,799,4,2,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,0,109
3,-9.071459,19.484619,-1.359802,1.584012,1.520616,-0.038735,801,1,2,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,0,109
4,3.386109,28.154275,-1.145692,0.035855,-1.227388,-0.713803,802,0,1,3.373729,28.175323,-1.134802,5.493111,1.930805,2.056836,1.514860,1.574333e+09,0,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,9.338509,-1.785875,-0.985730,-2.249014,-4.153894,0.002551,910,18,2,9.321332,-1.953993,-0.953376,5.736830,1.575032,1.091974,1.534059,1.574333e+09,747,109
3535,-5.076873,26.067745,-1.075473,1.118272,1.836733,0.026609,904,18,2,-5.123750,26.137417,-1.142295,2.748649,1.077847,1.625816,-0.453487,1.574333e+09,748,109
3536,9.369946,-1.803644,-1.026089,-1.393256,-2.879025,-0.152909,910,18,2,9.537793,-1.553594,-1.061839,6.661997,2.535438,1.397671,1.302334,1.574333e+09,748,109
3537,-5.025482,26.242204,-1.080742,1.042467,1.891490,0.021974,904,18,2,-5.128264,26.316446,-1.150593,2.670343,1.157835,1.589266,-0.424405,1.574333e+09,749,109


In [56]:
# c=0
# d = hdf5data['Sequence'][str(c)]
# track = np.asarray(d['Tracks'])
# detection = np.asarray(d['Detections'])
# row = link_detections_tracks(detection, track)
# append_df = link_detections_tracks(detection, track)
# append_df['timestamp'] = 2

In [85]:
# c=0
# d = hdf5data['Sequence'][str(c)]
# track = np.asarray(d['Tracks'])
# detection = np.asarray(d['Detections'])

# coord_detections = [np.asarray(vals[0].tolist()) for vals in detection]
# coord_tracks = [np.asarray(vals[1].tolist()) for vals in track]
# dists = cdist(coord_tracks, coord_detections, 'euclidean')
# print(type(dists))

<class 'numpy.ndarray'>
